In [ ]:
import scipy.misc
from scipy import ndimage
import sys
import os
import glob
import json
from collections import defaultdict
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl
import requests
import random
from datetime import datetime
import pandas as pd
from copy import deepcopy

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras import backend as K
from keras.utils import np_utils
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.utils.training_utils import multi_gpu_model
import sklearn.cross_validation as crv
from sklearn.metrics import f1_score

%matplotlib inline

In [ ]:
def read_image(filename):
    img = cv2.imread(filename, 0)
    
    #print(img.shape)
    
    #plt.figure(figsize=(40,30))
    #plt.imshow(img)
    #plt.gray()
    #plt.show()
    
    return img


def get_group_names(directory_path=None):
    if directory_path==None:
        pics_list = glob.glob("*.tif")
    else:
        pics_list = glob.glob(directory_path + "/" + "*.tif")
        
    pics_list_new = []
    
    for pics_name in pics_list:
        pics_list_new.append(pics_name[:-5])
        groups = list(set(pics_list_new))
        groups.sort()
        
    return groups 


def merge_rgb(group_name):
    
    r = read_image(group_name+str(1)+".tif")
    g = read_image(group_name+str(2)+".tif")
    b = read_image(group_name+str(3)+".tif")
        
    g = cv2.resize(g, (r.shape[1], r.shape[0]))
    b = cv2.resize(g, (r.shape[1], r.shape[0]))
        
    r = r.reshape(r.shape[0], r.shape[1], 1)
    g = g.reshape(g.shape[0], g.shape[1], 1)
    b = b.reshape(b.shape[0], b.shape[1], 1)
        
    merged = np.concatenate([r,g,b],axis=2)
    
    cv2.imwrite(group_name+"merged.tif", merged)


def main():
    groups = get_group_names()
    for group in groups:
        merge_rgb(group)

In [ ]:
main()

In [ ]:
! ls

In [ ]:
! mkdir condition_1
! mkdir condition_2
! mkdir condition_3
! mkdir control_1
! mkdir control_2
! mkdir control_3
! mv condition*1-merged.tif condition_1
! mv condition*2-merged.tif condition_2
! mv condition*3-merged.tif condition_3
! mv control*1-merged.tif control_1
! mv control*2-merged.tif control_2
! mv control*3-merged.tif control_3

In [ ]:
# 各種パラメータ

filenamewithdate = "directory_name"

path = "/path/to/directory/"

directory_path_0 = path+filenamewithdate+"/control_1/"
directory_path_1 = path+filenamewithdate+"/control_2/"
directory_path_2 = path+filenamewithdate+"/control_3/"
directory_path_3 = path+filenamewithdate+"/condition_1/"
directory_path_4 = path+filenamewithdate+"/condition_2/"  
directory_path_5 = path+filenamewithdate+"/condition_3/"

pic = 0             # pre_checkで何枚目の画像を表示するかを指定。デフォルトでは0で設定（1枚目）

binimg_thred = 80   # 2値化を行う場合の、ピクセル値を全体の何%の部分で2つに分けるかを設定。
                    # 位相差画像で色が黒い部分を検出したい場合はFluoroオプションをFalseとし、
                    # binimg_thredを1-10程度で調節する（暗いところを検出する）。
                    # 免疫染色画像で明るい部分を検出したい場合はFluoroオプションをTrueとし、
                    # binimg_thredを80-99程度で調節する（明るいところを検出するようになる）。

n_chan=3
                
chs = 0             # 2値化の際に使用するチャンネル（0:赤,1:緑,2:青）

fluoro = True      # 位相差画像で色が黒い部分を検出したい場合はFluoroオプションをFalseとし、
                    # 免疫染色画像で明るい部分を検出したい場合はFluoroオプションをTrueに設定する。
    
min_area = 1000       # 細胞と認識する最小面積

max_area = 99999999

scale_v = 75        # 切り抜く歳の画像の高さ/2 (px)

scale_h = 75        # 切り抜く際の画像の幅/2 (px)

In [ ]:
def making_pics_list(directory_path):
                
    path = directory_path + "*"  
    filenames = glob.glob(path)
    
    return filenames


def cell_crop_single(single_img, binimg_thred = 80., min_area=2000, max_area=4000, scale_v=75, scale_h=75, chs=0, fluoro=True):
    
    cells = np.empty((0, scale_v*2, scale_h*2, 3))
    
    img = single_img.astype(np.uint8)
    img_chs = cv2.split(img)
    img_preprocessed = cv2.GaussianBlur(img_chs[chs],(5,5),0)
    if fluoro==False:
        binimg = (img_preprocessed < np.percentile(img_preprocessed, binimg_thred))
        binimg = binimg.astype(np.uint8)
    else:
        binimg = (img_preprocessed > np.percentile(img_preprocessed, binimg_thred))
        binimg = binimg.astype(np.uint8)

    img_, contours, _ = cv2.findContours(binimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    arr=[]
    
    start=np.empty((0,2))
    start=np.append(start,np.array([[0, 0]]),axis=0)
    
    for j in contours:
        if cv2.contourArea(j)<min_area:
            continue
        if cv2.contourArea(j)>max_area:
            continue
        x_=0
        y_=0
        for k in j:
            x_ += k[0][0]
            y_ += k[0][1]
        arr.append([x_/len(j), y_/len(j)])
    arr = np.array(arr)
    
    
    for j in range(len(arr)):
    
        if (arr[j][1] < scale_v) or (arr[j][1] > img.shape[0]-scale_v) or (arr[j][0] < scale_h) or (arr[j][0] > img.shape[1]-scale_h):
            continue 
        
        top = int(arr[j][1])-scale_v
        bottom = int(arr[j][1])+scale_v
    
        left = int(arr[j][0])-scale_h
        right = int(arr[j][0])+scale_h
    
        if left < 0:
            left = 0
            right = scale_h*2
        if right > img.shape[1]:
            right = img.shape[1]
            left = img.shape[1]-scale_h*2
    
        if top < 0:
            top = 0
            bottom = scale_v*2
        if bottom > img.shape[0]:
            bottom = img.shape[0]
            top = img.shape[0]-scale_v*2      
                
        img_crop = np.array(img[top:bottom,left:right]).reshape(scale_v*2, scale_h*2, 3).astype(np.uint8)
        img_chs = cv2.split(img_crop)
        img_preprocessed = cv2.GaussianBlur(img_chs[chs],(5,5),0)
            
        if fluoro==False:
            binimg = (img_preprocessed < np.percentile(img_preprocessed, binimg_thred))
            binimg = binimg.astype(np.uint8)
        else:
            binimg = (img_preprocessed > np.percentile(img_preprocessed, binimg_thred))
            binimg = binimg.astype(np.uint8)

        img_, contours, _ = cv2.findContours(binimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            
        contourArea = []
            
        for j in contours:
            contourArea.append(cv2.contourArea(j))
        contourArea_sum = sum(contourArea)
        if contourArea_sum<min_area:
            continue
    
        cells = np.append(cells,np.array(img[top:bottom,left:right]).reshape(1,scale_v*2, scale_h*2, 3),axis=0)

    print("cropped_cell_count:", cells.shape[0])
    
    fig = plt.figure(figsize=(12, 400))
    
    cropped_cell_count = cells.shape[0]
    
    if cropped_cell_count>2000:
        cropped_cell_count = 2000

    for i in range(cropped_cell_count):
        
        ax_cell = fig.add_subplot(200, 10, i + 1, xticks=[], yticks=[])
        ax_cell.imshow(cells[i].reshape((scale_v*2, scale_h*2, 3)).astype(np.uint8))

    plt.show()
    
    return cells    
    

def cell_crop_from_each_pic(filenames):
    
    global binimg_thred, min_area, scale_v, scale_h, chs, fluoro
    
    total_cells = np.empty((0, scale_v*2, scale_h*2, 3))
    
    for filename in filenames:
        print("filename:", filename)
        img = scipy.misc.imread(filename)
        height, width, chan = img.shape
        assert chan == 3
        cells = cell_crop_single(img, binimg_thred=binimg_thred, min_area=min_area, max_area=max_area, scale_v=scale_v, scale_h=scale_h, chs=chs, fluoro=fluoro)
        total_cells = np.append(total_cells,cells,axis=0)
    
    print("total_cropped_cell_count:", total_cells.shape[0])    
    
    return total_cells  


def pre_check(X, pic=0, binimg_thred = 80., chs=0, fluoro=True):

    img = X[pic].astype(np.uint8)
    
    print("")
    print("##### original picture #####")
    plt.imshow(img)
    plt.show()

    img_chs = cv2.split(img)
    img_preprocessed = cv2.GaussianBlur(img_chs[chs],(5,5),0)
    if fluoro==False:
        binimg = (img_preprocessed < np.percentile(img_preprocessed, binimg_thred))
        binimg = binimg.astype(np.uint8)
    else:
        binimg = (img_preprocessed > np.percentile(img_preprocessed, binimg_thred))
        binimg = binimg.astype(np.uint8)

    print("")
    print("##### post binarization #####")
    plt.imshow(binimg)
    plt.colorbar()
    plt.show()


def preprocess_input(x0):
    return ((x0/255.)-0.5)*2.
    


def cell_crop_and_select(directory_path):
    
    filenames = making_pics_list(directory_path)
    cells = cell_crop_from_each_pic(filenames)
    
    return cells


def line_notify(message):
    
    url = "https://notify-api.line.me/api/notify"
    token = "UaZd8ZlZz4i2FidTS8cK6j9J4oqc808CRk7lzHQ84qn"
    headers = {"Authorization" : "Bearer "+ token}
    message = "\n" + message
    payload = {"message" :  message}

    r = requests.post(url ,headers = headers ,params=payload)

In [ ]:
pics_0 = cell_crop_and_select(directory_path_0)
pics_1 = cell_crop_and_select(directory_path_1)
pics_2 = cell_crop_and_select(directory_path_2)
pics_3 = cell_crop_and_select(directory_path_3)
pics_4 = cell_crop_and_select(directory_path_4)
pics_5 = cell_crop_and_select(directory_path_5)

In [ ]:
print(pics_0.shape)
print(pics_1.shape)
print(pics_2.shape)
print(pics_3.shape)
print(pics_4.shape)
print(pics_5.shape)

In [ ]:
np.save("pics_control_1_"+filenamewithdate+".npy", pics_0)
np.save("pics_control_2_"+filenamewithdate+".npy", pics_1)
np.save("pics_control_3_"+filenamewithdate+".npy", pics_2)
np.save("pics_condition_1_"+filenamewithdate+".npy", pics_3)
np.save("pics_condition_2_"+filenamewithdate+".npy", pics_4)
np.save("pics_condition_3_"+filenamewithdate+".npy", pics_5)

In [ ]:
# Deeplerning用各種パラメータ

gpu_count = 4
n_chan=3

scale_v = 75        # 切り抜く歳の画像の高さ/2 (px)
scale_h = 75        # 切り抜く際の画像の幅/2 (px)

nb_epoch = 2
nb_classes = 2
batch_size = 128

multi_gpu = False
use_fit = True
data_augmentation_scale ="small"  # full, mid, or small

In [ ]:
# データ読み込みとリスト作成

filenamewithdate = "directory_name"
path = "/path/to/directory/"

pics_control_1 = np.load(path+"/"+filenamewithdate+"/pics_control_1_"+filenamewithdate+".npy")
pics_condition_1 = np.load(path+"/"+filenamewithdate+"/pics_condition_1_"+filenamewithdate+".npy")
pics_control_2 = np.load(path+"/"+filenamewithdate+"/pics_control_2_"+filenamewithdate+".npy")
pics_condition_2 = np.load(path+"/"+filenamewithdate+"/pics_condition_2_"+filenamewithdate+".npy")
pics_control_3 = np.load(path+"/"+filenamewithdate+"/pics_control_3_"+filenamewithdate+".npy")
pics_condition_3 = np.load(path+"/"+filenamewithdate+"/pics_condition_3_"+filenamewithdate+".npy")

print(pics_control_1.shape)
print(pics_condition_1.shape)
print(pics_control_2.shape)
print(pics_condition_2.shape)
print(pics_control_3.shape)
print(pics_condition_3.shape)


control_list = [pics_control_1, pics_control_2, pics_control_3]
condition_list = [pics_condition_1, pics_condi_2, pics_ang2_3]
et1_list = [pics_et1_1, pics_et1_2, pics_et1_3]

In [ ]:
def data_setting(condition_1, condition_2):
    
    if type(condition_1)==list and type(condition_2)==list:
    
        X_tmp_1 = np.empty((0,scale_v*2,scale_h*2,3))
        X_tmp_2 = np.empty((0,scale_v*2,scale_h*2,3))
    
        for i in range(len(condition_1)):
        
            X_tmp_1 = np.concatenate((X_tmp_1, condition_1[i]),axis=0)
    
        for i in range(len(condition_2)):
        
            X_tmp_2 = np.concatenate((X_tmp_2, condition_2[i]),axis=0)    
    
        X = np.concatenate((X_tmp_1, X_tmp_2),axis=0)
        y = np.concatenate((np.tile(np.array([[0]]),(X_tmp_1.shape[0],1)),
                             np.tile(np.array([[1]]),(X_tmp_2.shape[0],1))
                            ),axis=0)
    
        X = preprocess_input(X)

        X = X.reshape(X.shape[0], scale_v*2, scale_h*2, n_chan)
        Y = np_utils.to_categorical(y, 2)
        
    else:
        
        X = np.concatenate((condition_1, condition_2),axis=0)
        y = np.concatenate((np.tile(np.array([[0]]),(condition_1.shape[0],1)),
                             np.tile(np.array([[1]]),(condition_2.shape[0],1))
                            ),axis=0)
    
        X = preprocess_input(X)
        X = X.reshape(X.shape[0], scale_v*2, scale_h*2, n_chan)
    
        Y = np_utils.to_categorical(y, 2)
        
    np.random.seed(0)
    np.random.shuffle(X)
    np.random.seed(0)
    np.random.shuffle(Y)

    return X, Y


def data_augmentation(X_train, X_test, Y_train, Y_test, scale=data_augmentation_scale):

    if scale=="full":
    
        train_X_00 = X_train[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :] 
 
        train_X_flip = X_train[:, :, ::-1, :] 
 
        test_X_00 = X_test[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :] 
 
        train_X_01 = X_train[:, 0:(scale_v*2 - 30), 0:(scale_h*2 - 30), :] 
        train_X_02 = X_train[:, 30:scale_v*2, 30:scale_h*2, :] 
        train_X_03 = X_train[:, 30:scale_v*2, 0:(scale_h*2 - 30), :] 
        train_X_04 = X_train[:, 0:(scale_v*2 - 30), 30:scale_h*2, :] 
 
        train_X_05 = train_X_flip[:, 0:(scale_v*2 - 30), 0:(scale_h*2 - 30), :] 
        train_X_06 = train_X_flip[:, 30:scale_v*2, 30:scale_h*2, :] 
        train_X_07 = train_X_flip[:, 30:scale_v*2, 0:(scale_h*2 - 30), :] 
        train_X_08 = train_X_flip[:, 0:(scale_v*2 - 30), 30:scale_h*2, :] 
        train_X_09 = train_X_flip[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :]

        train_X_rotate = np.empty((0,scale_v*2-30,scale_h*2-30,3))

        for i in [train_X_00,train_X_01,train_X_02,train_X_03,train_X_04,train_X_05,train_X_06,train_X_07,train_X_08,train_X_09]:
            for j in [90,180,270]:
                train_X_rotate = np.append(train_X_rotate, ndimage.rotate(i, j, axes=(1,2), reshape=False, mode='nearest'), axis=0) 
    
        X_train = np.concatenate((train_X_00,train_X_01,train_X_02,train_X_03,train_X_04,train_X_05,train_X_06,train_X_07,train_X_08,train_X_09,train_X_rotate),axis=0)
        
    elif scale=="mid":
        
        train_X_00 = X_train[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :] 
 
        test_X_00 = X_test[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :] 

        train_X_rotate = np.empty((0,scale_v*2-30,scale_h*2-30,3))

        for i in [train_X_00]:
            for j in [90,180,270]:
                train_X_rotate = np.append(train_X_rotate, ndimage.rotate(i, j, axes=(1,2), reshape=False, mode='nearest'), axis=0) 
    
        X_train = np.concatenate((train_X_00,train_X_rotate),axis=0)
        
    elif scale=="small":
        
        train_X_00 = X_train[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :] 
 
        test_X_00 = X_test[:, 15:(scale_v*2 - 15), 15:(scale_h*2 - 15), :] 

        X_train = train_X_00
    
    Y_train = np.tile(Y_train, (int(X_train.shape[0]/train_X_00.shape[0]), 1)) 
    X_test = test_X_00
    
    print("X_train data count:", X_train.shape[0])
    print("X_test data count:", X_test.shape[0])
 
    return X_train, X_test, Y_train, Y_test


def build_model_xception(nb_classes=2, multi_gpu=multi_gpu):
    
    keras.backend.clear_session()  
    
    if multi_gpu==False:
    
        base_model = Xception(weights='imagenet', include_top=False)

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(nb_classes, activation='softmax')(x)

        model_ = Model(inputs=base_model.input, outputs=predictions)
        model = model_
    
    if multi_gpu==True:
    
        with tf.device("/cpu:0"):
            base_model = Xception(weights='imagenet', include_top=False)
    
            x = base_model.output
            x = GlobalAveragePooling2D()(x)
            x = Dense(1024, activation='relu')(x)
            predictions = Dense(nb_classes, activation='softmax')(x)

            model_ = Model(inputs=base_model.input, outputs=predictions)
        
        model = multi_gpu_model(model_, gpus=gpu_count)
    
    for layer in base_model.layers:
        layer.trainable = True

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])
    
    return model


def plot_history(history):
    
    plt.plot(history.history['acc'],"o-",label="accuracy")
    plt.plot(history.history['val_acc'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.ylim([0,1])
    
    plt.savefig(datetime.now().strftime('%Y%m%d%H%M%S')+"acc.png")
    
    plt.show()

    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='upper right')
    
    plt.savefig(datetime.now().strftime('%Y%m%d%H%M%S')+"loss.png")
    
    plt.show()
    
    df = pd.DataFrame({ 'accuracy' : history.history['acc'],
                        'val_acc' : history.history['val_acc'],
                        'loss' : history.history['loss'],
                        'val_loss' : history.history['val_loss']})
    
    df.to_csv(datetime.now().strftime('%Y%m%d%H%M%S')+".csv")
        


def do_xception(X_train, X_test, Y_train, Y_test, multi_gpu=multi_gpu, use_fit=use_fit):
    
    global batch_size, gpu_count, nb_epoch, model, history_x
    
    if multi_gpu==False and use_fit==False:
    
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=45,
            width_shift_range=0.25,
            height_shift_range=0.25,
            horizontal_flip=False,
            vertical_flip=False,
            fill_mode='nearest')
    
        history_x = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size, shuffle=True),
        steps_per_epoch=X_train.shape[0]//batch_size,
        epochs=nb_epoch,
        validation_data=datagen.flow(X_test, Y_test, batch_size=batch_size),
        validation_steps=X_test.shape[0]//batch_size)
    
    elif multi_gpu==True and use_fit==False:
        
        datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=0,
            width_shift_range=0,
            height_shift_range=0,
            horizontal_flip=False,
            vertical_flip=False,
            fill_mode='nearest')
    
        history_x = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size*gpu_count, shuffle=True),
        steps_per_epoch=X_train.shape[0]//(batch_size*gpu_count),
        epochs=nb_epoch,
        validation_data=datagen.flow(X_test, Y_test, batch_size=batch_size*gpu_count),
        validation_steps=X_test.shape[0]//(batch_size*gpu_count),
        workers=8,
        max_queue_size=16,
        use_multiprocessing=True)
    
    elif multi_gpu==True and use_fit==True:
        
        history_x = model.fit(x=X_train, y=Y_train, batch_size=(batch_size*gpu_count), shuffle=True,
        epochs=nb_epoch,
        validation_data=(X_test, Y_test))
        
    elif multi_gpu==False and use_fit==True:
        
        history_x = model.fit(x=X_train, y=Y_train, batch_size=batch_size, shuffle=True,
        epochs=nb_epoch,
        validation_data=(X_test, Y_test))


def binary_xception(condition_1, condition_2, test_1, test_2, multi_gpu=multi_gpu, use_fit=use_fit, scale=data_augmentation_scale):
    
    global nb_classes, batch_size, gpu_count, model, history_x
    
    X_train, Y_train = data_setting(condition_1, condition_2)
    X_test, Y_test = data_setting(test_1, test_2)
    X_train, X_test, Y_train, Y_test = data_augmentation(X_train, X_test, Y_train, Y_test, scale=scale)
    
    model = build_model_xception(nb_classes=nb_classes, multi_gpu=multi_gpu)
    
    do_xception(X_train, X_test, Y_train, Y_test, multi_gpu=multi_gpu, use_fit=use_fit)
    
    Y_pred = model.predict(X_test, batch_size=batch_size)
    y_pred = np.argmax(Y_pred, axis=1)
    print('final accuracy:',f1_score(np.argmax(Y_test,1), y_pred, average='macro'))
    
    plot_history(history_x)  


def cross_validation(control_list, subject_list, multi_gpu=multi_gpu, use_fit=use_fit, scale=data_augmentation_scale):
    
    global nb_classes, batch_size, gpu_count, model, history_x
    
    for i in range(len(control_list)):
        
        control_list_tmp = deepcopy(control_list)
        subject_list_tmp = deepcopy(subject_list)
        
        control_target = control_list_tmp[i]
        subject_target = subject_list_tmp[i]
        control_list_tmp.pop(i)
        subject_list_tmp.pop(i)
        
        print("cross validation:", i)
        
        random.seed(0)
        np.random.seed(0)
        tf.set_random_seed(0)
        
        binary_xception(control_list_tmp, subject_list_tmp, control_target, subject_target, multi_gpu=multi_gpu, use_fit=use_fit, scale=scale)
   

In [ ]:
random.seed(0)
np.random.seed(0)
tf.set_random_seed(0)

In [ ]:
cross_validation(control_list, ang2_list)